In [1]:
import numpy as np
import pandas as pd
import warnings
import time
from scipy import signal
from scipy.signal import butter, lfilter
import matplotlib.pyplot as plt
import math
import pickle
import statistics
import random
from sklearn.metrics.pairwise import cosine_similarity
from math import log
from sklearn import preprocessing
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import sktime as sktime
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sktime.datasets import load_from_tsfile_to_dataframe
from sktime.alignment.dtw_python import AlignerDTWfromDist
import os
import sys
%matplotlib inline
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sktime.classification.dictionary_based import MUSE
import warnings
warnings.filterwarnings("ignore")

/Users/abhidnyapatharkar/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Dictionary saved to Final_hb_sepsis_MUSE_normalized_CV.pickle


### Load normal and abnormal datasets

In [2]:
final_normal_df = pd.read_pickle('normal_public_sepsis.pkl')
final_abnormal_df = pd.read_pickle('abnormal_public_sepsis.pkl')

,Subject,HR,MAP,DBP,SBP,Resp
0,1,75.0,63.0,46.0,108.0,13.0
1,1,75.0,63.0,46.0,108.0,13.0
2,1,80.0,75.0,55.0,123.0,19.0
3,1,78.0,58.0,44.0,103.0,22.0
4,1,82.0,76.0,54.0,126.0,24.0


,Subject,HR,MAP,DBP,SBP,Resp
0,1,103.0,80.0,64.0,112.0,20.0
1,1,111.0,93.5,80.5,131.0,20.0
2,1,106.0,87.0,77.0,114.0,20.0
3,1,107.0,62.0,54.0,85.0,20.0
4,1,106.0,80.0,70.0,109.0,23.0


In [3]:
abnormal_indices = (final_abnormal_df['Subject']).unique()
abnormal_no = (final_abnormal_df['Subject']).nunique()

In [4]:
# Specify the file path of the saved pkl file
file_path = 'final_bins.pkl'

# Load the dictionary from the pkl file
with open(file_path, 'rb') as file:
    final_numbers_dictionary = pickle.load(file)
# Now, 'loaded_dictionary' contains the dictionary loaded from the pkl file
display("Dictionary loaded from pickle file:")

'Dictionary loaded from pickle file:'

{'sample_numbers_list_0.0_to_0.1': [112,
  360,
  143,
  415,
  413,
  23,
  374,
  32,
  34,
  42,
  68,
  90,
  119,
  121,
  137,
  178,
  180,
  196,
  214,
  241,
  242,
  249,
  257,
  265,
  277,
  279,
  315,
  325,
  340,
  357,
  365,
  369,
  406,
  407,
  426,
  429,
  441,
  455,
  458,
  470,
  493],
 'sample_numbers_list_0.1_to_0.2': [14,
  288,
  20,
  33,
  35,
  51,
  52,
  60,
  63,
  67,
  73,
  144,
  167,
  176,
  205,
  228,
  235,
  250,
  296,
  313,
  323,
  337,
  362,
  396,
  409,
  419,
  440,
  464,
  467,
  486,
  489],
 'sample_numbers_list_0.2_to_0.3': [378,
  280,
  50,
  3,
  5,
  41,
  43,
  70,
  117,
  123,
  139,
  164,
  220,
  225,
  244,
  247,
  262,
  289,
  320,
  334,
  344,
  350,
  363,
  381,
  393,
  394,
  417,
  444,
  474,
  480,
  487,
  496],
 'sample_numbers_list_0.3_to_0.4': [72,
  457,
  7,
  27,
  30,
  69,
  79,
  100,
  138,
  150,
  160,
  163,
  204,
  232,
  245,
  300,
  331,
  336,
  341,
  352,
  377,
  402,
  408,
  4

In [5]:
dictionary_keys = final_numbers_dictionary.keys()
keys_list = list(dictionary_keys)

### Function to find the length of the time series

In [6]:
# Define a function to z-normalize a column
def z_normalize(column):
    mean = column.mean()
    std = column.std()
    z_normalized = (column - mean) / std
    return z_normalized

In [7]:
def time_series_length_finding(df):
    train_1_abnormal = df.copy()
    train_1_ab_sub = (train_1_abnormal['Subject'].unique()).tolist()
    row_count = []
    for i in train_1_ab_sub:
        train_1_abnormal_new = train_1_abnormal[(train_1_abnormal['Subject']== i)]
        no_rows = train_1_abnormal_new.shape[0]
        row_count.append(no_rows)
    display("Avg length:",statistics.mean(row_count))
    avg_len_ts = int(statistics.mean(row_count))
    return(avg_len_ts)

In [8]:
def preprocessing_normal_df(df,desired_length):
    train_2_normal = df.copy()
    df_train_3 = pd.DataFrame(columns = train_2_normal.columns)
    num_dim = df_train_3.shape[1]
    display(num_dim)
    df_train_3.columns = range(len(df_train_3.columns))
    train_2_sub = (train_2_normal['Subject'].unique()).tolist()
    m = 0
    for i in train_2_sub:
        train_2_normal_new = train_2_normal[(train_2_normal['Subject']== i)]
        df_row_count = train_2_normal_new.shape[0]
        if df_row_count < desired_length:
            rows_to_add = desired_length - df_row_count
            train_2_normal_new.tail()
            last_row = train_2_normal_new.ffill().iloc[[-1]]  # Extract the last row
            new_rows_df = pd.concat([last_row] * rows_to_add, ignore_index=True)
            train_2_normal_initial = pd.concat([train_2_normal_new, new_rows_df], ignore_index=True)
            train_2_normal_initial_no_sub = train_2_normal_initial.drop(columns='Subject')
            train_2_normal_final = z_normalize(train_2_normal_initial_no_sub)
            train_2_normal_final.insert(0, 'Subject', i)
        else:
            train_2_normal_initial = train_2_normal_new.iloc[:desired_length,:]
            train_2_normal_initial_no_sub = train_2_normal_initial.drop(columns='Subject')
            train_2_normal_final = z_normalize(train_2_normal_initial_no_sub)
            train_2_normal_final.insert(0, 'Subject', i)

        train_2_normal_final.columns = range(len(train_2_normal_final.columns))

        df_train_3.at[m,0] = i
        j = 0
        while j < num_dim:
            df_train_3.at[m,j] = train_2_normal_final[j]
            j += 1
        m += 1
    df_train_3['class'] = 'normal'
    df_normal_baseline = df_train_3.iloc[:,1:]
    return(df_normal_baseline)

In [9]:
def preprocessing_dataframe(df,desired_length):
    train_1_abnormal = df.copy()
    df_train_2 = pd.DataFrame(columns = train_1_abnormal.columns)
    num_dim_ab = df_train_2.shape[1]
    df_train_2.columns = range(len(df_train_2.columns))
    train_1_ab_sub = (train_1_abnormal['Subject'].unique()).tolist()
    m = 0
    for i in train_1_ab_sub:
        train_1_abnormal_new = train_1_abnormal[(train_1_abnormal['Subject']== i)]
        df_row_count = train_1_abnormal_new.shape[0]
        if df_row_count < desired_length:
            rows_to_add = desired_length - df_row_count
            last_row = train_1_abnormal_new.ffill().iloc[[-1]]  # Extract the last row
            new_rows_df = pd.concat([last_row] * rows_to_add, ignore_index=True)
            train_1_abnormal_initial = pd.concat([train_1_abnormal_new, new_rows_df], ignore_index=True)
            train_1_abnormal_initial_no_sub = train_1_abnormal_initial.drop(columns='Subject')
            train_1_abnormal_final = z_normalize(train_1_abnormal_initial_no_sub)
            train_1_abnormal_final.insert(0, 'Subject', i)
        else:
            train_1_abnormal_initial = train_1_abnormal_new.iloc[:desired_length,:]
            train_1_abnormal_initial_no_sub = train_1_abnormal_initial.drop(columns='Subject')
            train_1_abnormal_final = z_normalize(train_1_abnormal_initial_no_sub)
            train_1_abnormal_final.insert(0, 'Subject', i)
        train_1_abnormal_final.columns = range(len(train_1_abnormal_final.columns))

        df_train_2.at[m,0] = i
        j = 0
        while j < num_dim_ab:
            df_train_2.at[m,j] = train_1_abnormal_final[j]
            j += 1
        m += 1
    df_train_2['class'] = 'abnormal'
    df_final = df_train_2.iloc[:,1:]
    return(df_final)

In [27]:
def ts_classifier(df_class_1):
    acc_mean_list = []
    prec_mean_list = []
    recall_mean_list = []
    f1_mean_list = []

    X = df_class_1.iloc[:,:-1]
    y = df_class_1.iloc[:,-1]

    for i in range(1, 31):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=i, stratify=y)
        
        accuracy_dict = {} 
        precision_dict = {}
        recall_dict = {}
        f1_dict = {}
        
        comb = 1
        for a in[2,4,6]:
            for b in [4,8]:
                clf = MUSE(window_inc=a,alphabet_size=b,use_first_order_differences=False, random_state=42)
                clf.fit(X_train, y_train)
                
                # Predict using the final model
                y_pred = clf.predict(X_test)

                pos_label = 'abnormal'

                acc = accuracy_score(y_test, y_pred)
                prec = precision_score(y_test, y_pred, pos_label=pos_label)
                recall = recall_score(y_test, y_pred, pos_label=pos_label)
                f1 = f1_score(y_test, y_pred, pos_label=pos_label)
                
                accuracy_dict[(a, b)] = acc
                precision_dict[(a, b)] = prec
                recall_dict[(a, b)] = recall
                f1_dict[(a, b)] = f1
                
                display("Completed comb",comb)
                comb += 1
        
        accuracy_max_dict = max(accuracy_dict.items(), key=lambda x: x[1])
        display("Accuracy Dictionary:",accuracy_dict)
        accuracy_max = accuracy_max_dict[1]

        max_a_b = accuracy_max_dict[0]
        display("(a, b) combination for maximum accuracy:",max_a_b)

        display('Accuracy: %f' % accuracy_max)
        acc_mean_list.append(accuracy_max)

        precision_max = precision_dict[max_a_b]
        display('Precision: %f' % precision_max)
        prec_mean_list.append(precision_max)

        recall_max = recall_dict[max_a_b]
        display('Recall: %f' % recall_max)
        recall_mean_list.append(recall_max)

        f1_score_max = f1_dict[max_a_b]
        display('f1_score: %f' % f1_score_max)
        f1_mean_list.append(f1_score_max)

        display("Completed", i)

    acc_mean = np.mean(acc_mean_list)
    acc_std_dev = np.std(acc_mean_list)
    display('Accuracy Mean: %f' % acc_mean)

    prec_mean = np.mean(prec_mean_list)
    prec_std_dev = np.std(prec_mean_list)

    recall_mean = np.mean(recall_mean_list)
    recall_std_dev = np.std(recall_mean_list)

    f1_mean = np.mean(f1_mean_list)
    f1_std_dev = np.std(f1_mean_list)
    display('F1 Mean: %f' % f1_mean)

    return (acc_mean, acc_std_dev, prec_mean, prec_std_dev, recall_mean, recall_std_dev, f1_mean, f1_std_dev)

In [28]:
# Keys to combine
key_to_combine1 = 'sample_numbers_list_0.0_to_0.1'
key_to_combine2 = 'sample_numbers_list_0.1_to_0.2'
# Create a new dictionary with combined values
hb_dict = {
    'hb_key': final_numbers_dictionary[key_to_combine1] + final_numbers_dictionary[key_to_combine2]
}

In [29]:
classifier_dict_hb = {}
key = 'hb_key'
normal_sample_numbers = []
normal_sample_numbers.extend(hb_dict[key])
normal_sample_length = len(normal_sample_numbers)
display(normal_sample_length)

if len(normal_sample_numbers) <= abnormal_no:
    df_normal = final_normal_df[final_normal_df['Subject'].isin(normal_sample_numbers)]
    display(df_normal.head(2))

    random.seed(42)
    abnormal_sample_numbers = random.sample(abnormal_indices.tolist(), normal_sample_length)
    df_abnormal = final_abnormal_df[final_abnormal_df['Subject'].isin(abnormal_sample_numbers)]
    display(df_abnormal.head(2))

    desired_length_for_ts = time_series_length_finding(df_abnormal)

    df_normal_preprocessed = preprocessing_normal_df(df_normal,desired_length_for_ts)
    df_abnormal_preprocessed = preprocessing_dataframe(df_abnormal,desired_length_for_ts)

    df_classifier = (pd.concat([df_normal_preprocessed,df_abnormal_preprocessed]).reset_index()).drop(columns='index')
    acc_mean,acc_std_dev,prec_mean,prec_std_dev,recall_mean,recall_std_dev,f1_mean,f1_std_dev = ts_classifier(df_classifier)

    classifier_dict_hb[key] = {'acc_mean': acc_mean,'acc_std_dev': acc_std_dev,'prec_mean': prec_mean,'prec_std_dev': prec_std_dev,'recall_mean': recall_mean,'recall_std_dev': recall_std_dev,'f1_mean': f1_mean,'f1_std_dev': f1_std_dev}

else:
    df_abnormal = final_abnormal_df

    random.seed(42)
    random_sample_numbers = random.sample(normal_sample_numbers, abnormal_no)
    df_normal = final_normal_df[final_normal_df['Subject'].isin(random_sample_numbers)]
    display(df_normal.head(2))

    desired_length_for_ts = time_series_length_finding(df_abnormal)

    df_normal_preprocessed = preprocessing_normal_df(df_normal,desired_length_for_ts)
    df_abnormal_preprocessed = preprocessing_dataframe(df_abnormal,desired_length_for_ts)

    df_classifier = (pd.concat([df_normal_preprocessed,df_abnormal_preprocessed]).reset_index()).drop(columns='index')
    acc_mean,acc_std_dev,prec_mean,prec_std_dev,recall_mean,recall_std_dev,f1_mean,f1_std_dev = ts_classifier(df_classifier)

    classifier_dict_hb[key] = {'acc_mean': acc_mean,'acc_std_dev': acc_std_dev,'prec_mean': prec_mean,'prec_std_dev': prec_std_dev,'recall_mean': recall_mean,'recall_std_dev': recall_std_dev,'f1_mean': f1_mean,'f1_std_dev': f1_std_dev}

72

,Subject,HR,MAP,DBP,SBP,Resp
0,14,58.0,71.5,49.0,123.5,11.0
1,14,75.0,80.0,58.0,136.0,12.0


,Subject,HR,MAP,DBP,SBP,Resp
0,2,77.0,102.0,74.0,161.0,18.5
1,2,78.0,102.0,74.0,129.0,18.0


'Avg length:'

106.48611111111111

...

'f1_score: 0.888889'

'Completed'

30

'Accuracy Mean: 0.948276'

'F1 Mean: 0.944741'

In [30]:
classifier_dict_hb

{'hb_key': {'acc_mean': 0.9482758620689656,
  'acc_std_dev': 0.03643877582562611,
  'prec_mean': 0.9831257631257632,
  'prec_std_dev': 0.03062936017056298,
  'recall_mean': 0.9117460317460317,
  'recall_std_dev': 0.06534672144436428,
  'f1_mean': 0.9447406752464222,
  'f1_std_dev': 0.04001346219230573}}

In [31]:
import pickle

# Specify the file path where you want to save the pickle file
pickle_file_path = 'Final_hb_sepsis_MUSE_normalized_CV.pickle'

# Save the dictionary to a pickle file
with open(pickle_file_path, 'wb') as file:
    pickle.dump(classifier_dict_hb, file)

print(f"Dictionary saved to {pickle_file_path}")

In [32]:
all_values = [value for sublist in final_numbers_dictionary.values() for value in sublist]
healthy_dict = {'healthy_key': all_values}

{'healthy_key': [112,
  360,
  143,
  415,
  413,
  23,
  374,
  32,
  34,
  42,
  68,
  90,
  119,
  121,
  137,
  178,
  180,
  196,
  214,
  241,
  242,
  249,
  257,
  265,
  277,
  279,
  315,
  325,
  340,
  357,
  365,
  369,
  406,
  407,
  426,
  429,
  441,
  455,
  458,
  470,
  493,
  14,
  288,
  20,
  33,
  35,
  51,
  52,
  60,
  63,
  67,
  73,
  144,
  167,
  176,
  205,
  228,
  235,
  250,
  296,
  313,
  323,
  337,
  362,
  396,
  409,
  419,
  440,
  464,
  467,
  486,
  489,
  378,
  280,
  50,
  3,
  5,
  41,
  43,
  70,
  117,
  123,
  139,
  164,
  220,
  225,
  244,
  247,
  262,
  289,
  320,
  334,
  344,
  350,
  363,
  381,
  393,
  394,
  417,
  444,
  474,
  480,
  487,
  496,
  72,
  457,
  7,
  27,
  30,
  69,
  79,
  100,
  138,
  150,
  160,
  163,
  204,
  232,
  245,
  300,
  331,
  336,
  341,
  352,
  377,
  402,
  408,
  435,
  481,
  500,
  380,
  473,
  194,
  6,
  21,
  36,
  37,
  77,
  103,
  109,
  125,
  128,
  132,
  135,
  149,
  187,


In [ ]:
classifier_dict_healthy = {}
key = 'healthy_key'
normal_sample_numbers = []
normal_sample_numbers.extend(healthy_dict[key])
normal_sample_length = len(normal_sample_numbers)
display(normal_sample_length)

if len(normal_sample_numbers) <= abnormal_no:
    df_normal = final_normal_df[final_normal_df['Subject'].isin(normal_sample_numbers)]
    display(df_normal.head())

    random.seed(42)
    abnormal_sample_numbers = random.sample(abnormal_indices.tolist(), normal_sample_length)
    df_abnormal = final_abnormal_df[final_abnormal_df['Subject'].isin(abnormal_sample_numbers)]
    display(df_abnormal.head())

    desired_length_for_ts = time_series_length_finding(df_abnormal)

    df_normal_preprocessed = preprocessing_normal_df(df_normal,desired_length_for_ts)
    df_abnormal_preprocessed = preprocessing_dataframe(df_abnormal,desired_length_for_ts)

    df_classifier = (pd.concat([df_normal_preprocessed,df_abnormal_preprocessed]).reset_index()).drop(columns='index')
    acc_mean,acc_std_dev,prec_mean,prec_std_dev,recall_mean,recall_std_dev,f1_mean,f1_std_dev = ts_classifier(df_classifier)

    classifier_dict_healthy[key] = {'acc_mean': acc_mean,'acc_std_dev': acc_std_dev,'prec_mean': prec_mean,'prec_std_dev': prec_std_dev,'recall_mean': recall_mean,'recall_std_dev': recall_std_dev,'f1_mean': f1_mean,'f1_std_dev': f1_std_dev}

else:
    df_abnormal = final_abnormal_df

    random.seed(42)
    random_sample_numbers = random.sample(normal_sample_numbers, abnormal_no)
    df_normal = final_normal_df[final_normal_df['Subject'].isin(random_sample_numbers)]
    display(df_normal)

    desired_length_for_ts = time_series_length_finding(df_abnormal)

    df_normal_preprocessed = preprocessing_normal_df(df_normal,desired_length_for_ts)
    df_abnormal_preprocessed = preprocessing_dataframe(df_abnormal,desired_length_for_ts)

    df_classifier = (pd.concat([df_normal_preprocessed,df_abnormal_preprocessed]).reset_index()).drop(columns='index')
    acc_mean,acc_std_dev,prec_mean,prec_std_dev,recall_mean,recall_std_dev,f1_mean,f1_std_dev = ts_classifier(df_classifier)

    classifier_dict_healthy[key] = {'acc_mean': acc_mean,'acc_std_dev': acc_std_dev,'prec_mean': prec_mean,'prec_std_dev': prec_std_dev,'recall_mean': recall_mean,'recall_std_dev': recall_std_dev,'f1_mean': f1_mean,'f1_std_dev': f1_std_dev}

502

,Subject,HR,MAP,DBP,SBP,Resp
0,11,90.0,70.0,53.0,111.0,24.0
1,11,90.0,70.0,53.0,111.0,24.0
2,11,90.0,80.0,60.0,121.0,24.0
3,11,81.0,67.0,52.0,99.0,12.0
4,11,83.0,66.0,51.0,99.0,14.0
...,...,...,...,...,...,...
52,506,104.0,89.0,69.0,136.0,28.0
53,506,100.0,83.0,65.0,121.0,19.0
54,506,91.0,74.0,59.0,102.0,14.0
55,506,90.0,66.0,61.0,101.0,16.0


'Avg length:'

108.59428571428572

6

...

'f1_score: 0.825397'

'Completed'

29

'Completed comb'

1

In [ ]:
classifier_dict_healthy

In [ ]:
import pickle

# Specify the file path where you want to save the pickle file
pickle_file_path = 'Final_healthy_sepsis_MUSE_normalized_CV.pickle'

# Save the dictionary to a pickle file
with open(pickle_file_path, 'wb') as file:
    pickle.dump(classifier_dict_healthy, file)

print(f"Dictionary saved to {pickle_file_path}")